In [1]:
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import hist
from hist import Hist
import coffea
from coffea.nanoevents import NanoEventsFactory, BaseSchema
import coffea.processor as processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import numpy as np
import mplhep as hep


In [2]:
fname = "/eos/home-b/bchitrod/HHH/NANOAOD/TRSM_XToHY_6b_M3_2000_M2_1100_NANOAOD.root" 

In [3]:
def closest(masses):
    delta = abs(125 - masses)
    closest_masses = ak.min(delta, axis=1)
    is_closest = (delta == closest_masses)
    return is_closest

def HbbvsQCD(fatjet):
    score = (fatjet.particleNetMD_Xbb/(fatjet.particleNetMD_Xbb+fatjet.particleNetMD_QCD))
    return score

In [4]:
def boosted(fname,oFile,processLabel="signal",eventsToRead=None):	
    events = NanoEventsFactory.from_root(fname,schemaclass=NanoAODSchema,metadata={"dataset": "testSignal"},entry_stop=eventsToRead).events()

    fatjets = events.FatJet
    
    #Fatjet cuts
    ptcut = 250
    etacut = 2.5
    mass_cut = [100,150]
    pNet_cut = 0.9105

    good_fatjets = fatjets[(fatjets.pt>ptcut) & (np.absolute(fatjets.eta)<etacut) & (fatjets.msoftdrop>=mass_cut[0]) & (fatjets.msoftdrop<=mass_cut[1])]
    pre_boosted_fatjets = good_fatjets[ak.num(good_fatjets, axis=1)> 2]
    pre_boosted_events = events[ak.num(good_fatjets, axis=1)> 2]
    #Btag cut applied at the end
    btag_boosted = pre_boosted_fatjets[HbbvsQCD(pre_boosted_fatjets)>=pNet_cut]
    btag_boosted_fatjets = btag_boosted[ak.num(btag_boosted, axis=1)> 2]
    btag_boosted_events = pre_boosted_events[ak.num(btag_boosted, axis=1)> 2]

    return btag_boosted_fatjets


In [5]:
def plotMassDist(fname,oFile,processLabel="signal",eventsToRead=None):

    good_boosted = boosted(fname,oFile,processLabel="Signal",eventsToRead=None)

    trijet_mass = (good_boosted[:,0]+good_boosted[:,1]+good_boosted[:,2]).mass
    #calc inv mass of trijets by lorentz v. sum of three leading jets

    j3_bin = hist.axis.Regular(label="Trijet Mass [GeV]", name="trijet_mass", bins=40, start=0, stop=6000)
    j3_cat = hist.axis.StrCategory(label='Trijets', name='trijet', categories=[processLabel])

    j3_hist = Hist(j3_bin, j3_cat)
    j3_hist.fill(trijet=processLabel, trijet_mass=trijet_mass)

    plt.style.use([hep.style.CMS])
    j3_hist.plot(color="black")
    hep.cms.text("Work in progress",loc=0)
    plt.ylabel("Event count",horizontalalignment='right', y=1.0)
    plt.legend()
    plt.savefig("MJJJ_btag_{0}.png".format(oFile))
    plt.cla()
    plt.clf()

    #-----MJJ calc and plotting-----#


    dijet1_mass = (good_boosted[:,0]+good_boosted[:,1]).mass
    #calc inv mass of first dijet combination

    dijet2_mass = (good_boosted[:,0]+good_boosted[:,2]).mass
    #calc inv mass of second dijet combination

    dijet3_mass = (good_boosted[:,1]+good_boosted[:,2]).mass
    #calc inv mass of third dijet combination

    j2_bin = hist.axis.Regular(label="Dijet Mass [GeV]", name="dijet_mass", bins=40, start=0, stop=4000)
    j2_cat = hist.axis.StrCategory(label='Dijets', name='dijet', categories=["12 Pair","13 Pair","23 Pair"])

    j2_hist = Hist(j2_bin, j2_cat)

    j2_hist.fill(dijet="12 Pair", dijet_mass=dijet1_mass)
    j2_hist.fill(dijet="13 Pair", dijet_mass=dijet2_mass)
    j2_hist.fill(dijet="23 Pair", dijet_mass=dijet3_mass)
    
    mjj12_vs_mjjj = Hist(j3_bin,j2_bin)
    mjj12_vs_mjjj.fill(dijet_mass=dijet1_mass,trijet_mass=trijet_mass)
    mjj13_vs_mjjj = Hist(j3_bin,j2_bin)
    mjj13_vs_mjjj.fill(dijet_mass=dijet2_mass,trijet_mass=trijet_mass)
    mjj23_vs_mjjj = Hist(j3_bin,j2_bin)
    mjj23_vs_mjjj.fill(dijet_mass=dijet3_mass,trijet_mass=trijet_mass)
    
    j2_hist.plot(stack=True,histtype='fill',ec="black",fc=["violet","skyblue","khaki"])
    hep.cms.text("Work in progress",loc=0)
    plt.ylabel("Event count",horizontalalignment='right', y=1.0)
    plt.legend()
    plt.savefig("MJJ_btag_{0}.png".format(oFile))
    print("Saved MJJ_btag_{0}.png".format(oFile))
    plt.cla()
    plt.clf()
    return mjj12_vs_mjjj,mjj13_vs_mjjj,mjj23_vs_mjjj 

In [6]:
oFile = "output"
mjj12_vs_mjjj, mjj13_vs_mjjj, mjj23_vs_mjjj = plotMassDist(fname,oFile,processLabel="signal",eventsToRead=None)


Saved MJJ_btag_output.png


<Figure size 720x720 with 0 Axes>

In [7]:
with uproot.recreate("output.root") as fout:
    fout[f"mjj12_vs_mjjj"] = mjj12_vs_mjjj
    fout[f"mjj13_vs_mjjj"] = mjj13_vs_mjjj
    fout[f"mjj23_vs_mjjj"] = mjj23_vs_mjjj
        
with uproot.open("output.root") as fin:
    print(fin.items())

[('mjj12_vs_mjjj;1', <TH2D (version 4) at 0x7f82806afc70>), ('mjj13_vs_mjjj;1', <TH2D (version 4) at 0x7f822bff14c0>), ('mjj23_vs_mjjj;1', <TH2D (version 4) at 0x7f820bd0a6a0>)]
